In [1]:
import json
import sys
from collections import Counter

In [2]:
ref_tree_path = '/misc/vlgscratch4/BowmanGroup/datasets/bert_trees/ud_eng_pud_with_type.json'

In [3]:
with open(ref_tree_path, 'r') as f_ref:
    ref_trees = json.load(f_ref)

In [4]:
for key in ref_trees:
    print(ref_trees[key])
    break

{'sentence': '“While much of the digital transition is unprecedented in the United States, the peaceful transition of power is not,” Obama special assistant Kori Schulman wrote in a blog post Monday.\n', 'dependencies': [[[1, 20], 'punct'], [[2, 9], 'mark'], [[3, 9], 'nsubj'], [[4, 7], 'case'], [[5, 7], 'det'], [[6, 7], 'amod'], [[7, 3], 'nmod'], [[8, 9], 'cop'], [[9, 20], 'advcl'], [[10, 13], 'case'], [[11, 13], 'det'], [[12, 13], 'compound'], [[13, 9], 'obl'], [[14, 20], 'punct'], [[15, 17], 'det'], [[16, 17], 'amod'], [[17, 20], 'nsubj'], [[18, 19], 'case'], [[19, 17], 'nmod'], [[20, 0], 'root'], [[21, 20], 'advmod'], [[22, 20], 'punct'], [[23, 20], 'punct'], [[24, 26], 'compound'], [[25, 26], 'amod'], [[26, 29], 'nsubj'], [[27, 26], 'flat'], [[28, 26], 'flat'], [[29, 20], 'parataxis'], [[30, 33], 'case'], [[31, 33], 'det'], [[32, 33], 'compound'], [[33, 29], 'obl'], [[34, 29], 'nmod:tmod'], [[35, 20], 'punct']], 'tokens': ['“', 'While', 'much', 'of', 'the', 'digital', 'transition',

In [5]:
from bertviz import attention, visualization
from bertviz.pytorch_pretrained_bert import BertModel, BertTokenizer
import numpy as np
import h5py
import torch
import random

bert_version = 'bert-large-uncased'
model = BertModel.from_pretrained(bert_version)
tokenizer = BertTokenizer.from_pretrained(bert_version)



Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [6]:
sentence=ref_trees[key]['sentence']
dependencies = ref_trees[key]['dependencies']


In [7]:
tokens = tokenizer.tokenize(sentence)
tokens_a_delim = ['[CLS]'] + tokens + ['[SEP]']
token_ids =  tokenizer.convert_tokens_to_ids(tokens_a_delim)
tokens_tensor = torch.tensor([token_ids])

In [8]:
ref_tokens=ref_trees[key]['tokens']

In [9]:
token_type_tensor = torch.LongTensor([[0] * len(tokens_a_delim)])
_, _, attn_data_list =  model(tokens_tensor, token_type_ids=token_type_tensor)
attn_tensor = torch.stack([attn_data['attn_probs'] for attn_data in attn_data_list])
attention = attn_tensor.data.numpy()

In [ ]:
wsj_path = '/misc/vlgscratch4/BowmanGroup/datasets/ptb_trees/ptb3-wsj-test.json'

In [ ]:
with open(wsj_path) as fwsj:
    wsj_data = json.load(fwsj)

In [ ]:
dependencies[0]

# Eval

In [ ]:
def get_token_map(raw_gold_tokens, raw_bert_tokens):
    gold_tokens = [strip_accents(s.lower()) for s in raw_gold_tokens]
    bert_tokens = [s.replace("##", "") for s in raw_bert_tokens[1:-1]]

    gold_i = 0
    bert_i = 0
    gold_result = []
    bert_result = []
    while True:
        if gold_i == len(gold_tokens) and bert_i == len(bert_tokens):
            break
        elif gold_tokens[gold_i] == bert_tokens[bert_i]:
            gold_result.append((gold_i,))
            bert_result.append((bert_i,))
            gold_i += 1
            bert_i += 1
        elif len(bert_tokens[bert_i]) != len(gold_tokens[gold_i]):
            sub_bert_i_ls = [bert_i]
            sub_gold_i_ls = [gold_i]
            sub_gold_token = gold_tokens[gold_i]
            sub_bert_token = bert_tokens[bert_i]
            bert_i += 1
            gold_i += 1
            while len(sub_bert_token) != len(sub_gold_token):
                if len(sub_bert_token) < len(sub_gold_token):
                    sub_bert_i_ls.append(bert_i)
                    sub_bert_token += bert_tokens[bert_i]
                    bert_i += 1
                else:
                    sub_gold_i_ls.append(gold_i)
                    sub_gold_token += gold_tokens[gold_i]
                    gold_i += 1
            assert sub_bert_token == sub_gold_token
            bert_result.append(tuple(sub_bert_i_ls))
            gold_result.append(tuple(sub_gold_i_ls))
        else:
            raise Exception
    return gold_result, bert_result

In [10]:
attention = np.squeeze(attention, 1)
layers, heads, _ , _ = attention.shape

In [20]:
def max_relations(attn_matrix):
    max_relations = attn_arr.argmax(axis=1)+1
    return list(zip(range(1,len(max_relations+1)),max_relations.tolist())) 
        

In [21]:
attn_arr = attention[1, 1, 1:-1, 1:-1]
max_rel=max_relations(attn_arr)

In [24]:
max_rel

[(1, 15),
 (2, 38),
 (3, 38),
 (4, 38),
 (5, 38),
 (6, 38),
 (7, 38),
 (8, 38),
 (9, 38),
 (10, 15),
 (11, 38),
 (12, 38),
 (13, 38),
 (14, 38),
 (15, 38),
 (16, 15),
 (17, 38),
 (18, 15),
 (19, 38),
 (20, 38),
 (21, 15),
 (22, 38),
 (23, 38),
 (24, 38),
 (25, 38),
 (26, 38),
 (27, 38),
 (28, 38),
 (29, 38),
 (30, 38),
 (31, 38),
 (32, 38),
 (33, 38),
 (34, 38),
 (35, 38),
 (36, 38),
 (37, 38)]

In [ ]:
def get_accuracies(dependencies, relation_types):
    value_list = []
    
    for ref_c in dependencies:
        if tuple(ref_c[0]) in sys_tree:
                correct_type[ref_c[1]] += 1

In [ ]:
def eval(ref_trees, sys_trees, mode='sys'):
    '''
        return:
            mean F1 score
    '''
    prec_list = []
    reca_list = []
    f1_list = []
    correct_type = Counter()
    total_type = Counter()
    assert len(ref_trees)==len(sys_trees), "ref and sys should have same number of sentences"

    for i in ref_trees:
        ref_tree = set(tuple(x[0]) for x in ref_trees[i]['dependencies'])

    assert len(ref_trees)==len(sys_trees), "ref and sys should have same number of sentences"


In [30]:
sys_tree_path="/misc/vlgscratch4/BowmanGroup/pmh330/LINGA_outputs/bert_large_ud/relations_bert_large__layer01__head01.json"
with open(sys_tree_path, 'r') as f_sys:
    sys_trees = json.load(f_sys)

In [31]:
for i in ref_trees:
    ref_tree = set(tuple(x[0]) for x in ref_trees[i]['dependencies'])
    sys_tree = set(tuple(x) for x in sys_trees[i]['dependencies'])
    break
ref_tree

{(1, 20),
 (2, 9),
 (3, 9),
 (4, 7),
 (5, 7),
 (6, 7),
 (7, 3),
 (8, 9),
 (9, 20),
 (10, 13),
 (11, 13),
 (12, 13),
 (13, 9),
 (14, 20),
 (15, 17),
 (16, 17),
 (17, 20),
 (18, 19),
 (19, 17),
 (20, 0),
 (21, 20),
 (22, 20),
 (23, 20),
 (24, 26),
 (25, 26),
 (26, 29),
 (27, 26),
 (28, 26),
 (29, 20),
 (30, 33),
 (31, 33),
 (32, 33),
 (33, 29),
 (34, 29),
 (35, 20)}

In [32]:
sys_tree

{(1, 5),
 (2, 14),
 (3, 14),
 (4, 15),
 (5, 15),
 (6, 10),
 (7, 5),
 (8, 5),
 (9, 15),
 (10, 15),
 (11, 15),
 (12, 5),
 (13, 5),
 (14, 14),
 (15, 15),
 (16, 5),
 (17, 5),
 (18, 15),
 (19, 28),
 (20, 10),
 (21, 5),
 (22, 14),
 (23, 15),
 (24, 5),
 (25, 5),
 (26, 28),
 (27, 5),
 (28, 14),
 (29, 5),
 (30, 10),
 (31, 15),
 (32, 5),
 (33, 5),
 (34, 5),
 (35, 15)}